In [1]:
from PIL import Image
import glob
import numpy as np
import os


def load_data(img_path, number_of_data):  
    # 가위 : 0, 바위 : 1, 보 : 2
    # 내가 만든 train 이미지는 총 314개이다.
    
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    
    imgs = np.zeros(number_of_data*img_size*img_size*color, dtype=np.int32).reshape(number_of_data, img_size, img_size, color)
    # 1-4. 딥러닝 네트워크 학습시키기에 나온 내용 (컬러 채널값(3)을 만드는 reshape를 실행해야한다.)
    # 스텝에서 첫 번째 레이어에 input_shape=(28,28,1)로 지정했던 것을 기억하시나요? 
    # 그런데 print(x_train.shape) 을 해보면,(60000, 28, 28) 로 채널수에 대한 정보가 없습니다. 
    # 따라서 (60000, 28, 28, 1) 로 만들어 주어야 합니다
    # 라고 적혀있네요. 
    # mnist에서는 흑백이라서 1이었지만, 지금 이미지는 컬러이기 때문에 3
    # (314, 28, 28, 3) : 28*28 컬러사진 314장
    
    labels = np.zeros(number_of_data, dtype=np.int32)
    # np.zeros() : 0으로 초기화 된 shape 차원의 ndarray 배열 객체를 반환
    # dtype=np.int32 : ndarray에 담긴 데이터 티입을 정수형 숫자로 지정
    # https://rfriend.tistory.com/285 에서 참고함

    idx=0
    
    for file in glob.iglob(img_path+'/scissor/*.jpg'):    
        # glob.iglob() :실제로 동시에 저장하지 않고 glob()과 같은 값을 산출하는 이터레이터(반복가능한 객체)를 반환
        # glob는 해당 디렉토리의 파일명을 리스트 형식으로 반환한다.
        # * 는 모든 파일을 지정
      
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1
    # 가위 전체 파일을 for문으로 반복해서 열어서, 0 으로 분류  
    
     

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    # 바위 전체 파일을 for문으로 반복해서 열어서, 0 으로 분류 
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
     # 보 전체 파일을 for문으로 반복해서 열어서, 0 으로 분류 
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.") 

    # idx가 0으로 시작해서 for문을 세 번 돌고나서 +1씩 쌓인 누적 숫자 = 이미지 파일의 갯수
                           
    return imgs, labels

In [2]:
image_dir_path = os.getenv("HOME")+'/aiffel/rock_scissor_paper/img/train' # train 데이터 폴더 전체 오픈

(x_train, y_train)=load_data(image_dir_path,314)

# 위에서 작성한 함수 load_data 함수에 넣어서 x_train은 imgs, y_train은 labels로 반환된다.
# imgs는 (314, 28, 28, 3)으로 reshape된 img 배열이다.
# labels -> 가위 : 0, 바위 : 1, 보 : 2

x_train_norm = x_train/255.0   
# 입력은 0~1 사이의 값으로 정규화 (rgb값이 0~255기 때문에 255로 나눠준다.)
# x 데이터만 정규화한다. y는 하지 않음


print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 314 입니다.
x_train shape: (314, 28, 28, 3)
y_train shape: (314,)


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# model을 직접 만들어 보세요.
# Hint! model의 입력/출력부에 특히 유의해 주세요. 가위바위보 데이터셋은 MNIST 데이터셋과 어떤 점이 달라졌나요?
# [[YOUR CODE]]

# 이 값일 때, 평가 정확도가 가장 높게 나왔음
n_channel_1=64
n_channel_2=128
n_dense=16

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(n_channel_1, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(n_channel_2, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 3200)              0         
_________________________________________________________________
dense (Dense)                (None, 16)                51216     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1

In [4]:
model.fit(x_train_norm, y_train, epochs=30, validation_split=0.3)
# train 데이터를 학습시킨다.
# validation_split 옵션으로, train데이터의 30%를 validation 용으로 분류.
# 나머지 70%의 자료로 학습한 후, vaildation 데이터로 중간 평가 한다.

Epoch 1/30
7/7 [==============================] - 3s 54ms/step - loss: 2.0331 - accuracy: 0.0000e+00 - val_loss: 1.6898 - val_accuracy: 0.0000e+00
Epoch 2/30
7/7 [==============================] - 0s 7ms/step - loss: 1.4865 - accuracy: 0.3699 - val_loss: 1.4745 - val_accuracy: 0.0000e+00
Epoch 3/30
7/7 [==============================] - 0s 6ms/step - loss: 1.2656 - accuracy: 0.3699 - val_loss: 1.1475 - val_accuracy: 0.0000e+00
Epoch 4/30
7/7 [==============================] - 0s 6ms/step - loss: 1.0432 - accuracy: 0.4932 - val_loss: 1.2136 - val_accuracy: 0.0000e+00
Epoch 5/30
7/7 [==============================] - 0s 6ms/step - loss: 0.9464 - accuracy: 0.5845 - val_loss: 1.7125 - val_accuracy: 0.0000e+00
Epoch 6/30
7/7 [==============================] - 0s 6ms/step - loss: 0.8364 - accuracy: 0.7397 - val_loss: 1.4665 - val_accuracy: 0.0105
Epoch 7/30
7/7 [==============================] - 0s 6ms/step - loss: 0.7642 - accuracy: 0.7352 - val_loss: 1.6637 - val_accuracy: 0.0000e+00
Epoch

In [5]:
# 슬기님이 주신 테스트 데이터

image_test_dir_path = os.getenv("HOME")+'/aiffel/rock_scissor_paper/img/test'

(x_test, y_test)=load_data(image_test_dir_path, 334)
x_test_norm = x_test/255.0  

print("x_test shape: {}".format(x_test.shape))
print("y_test shape: {}".format(y_test.shape))

학습데이터(x_train)의 이미지 개수는 334 입니다.
x_test shape: (334, 28, 28, 3)
y_test shape: (334,)


In [6]:
# 모델 평가
model.evaluate(x_test_norm, y_test)

11/11 [==============================] - 0s 6ms/step - loss: 15.7398 - accuracy: 0.5389


[15.739775657653809, 0.538922131061554]

In [7]:
print(x_test_norm[0])

[[[0.57647059 0.6        0.6       ]
  [0.57647059 0.6        0.6       ]
  [0.57647059 0.59215686 0.60392157]
  ...
  [0.36078431 0.38039216 0.39215686]
  [0.34509804 0.36470588 0.37647059]
  [0.34509804 0.35686275 0.37647059]]

 [[0.62352941 0.64705882 0.64705882]
  [0.60784314 0.63137255 0.63137255]
  [0.59215686 0.60784314 0.61176471]
  ...
  [0.36862745 0.38823529 0.4       ]
  [0.35686275 0.37647059 0.38823529]
  [0.35294118 0.36470588 0.38431373]]

 [[0.62745098 0.65098039 0.65098039]
  [0.63137255 0.65490196 0.65490196]
  [0.63921569 0.65490196 0.65882353]
  ...
  [0.37647059 0.39607843 0.40784314]
  [0.36862745 0.38431373 0.39607843]
  [0.36078431 0.37647059 0.38823529]]

 ...

 [[0.3372549  0.35686275 0.36862745]
  [0.90196078 0.93333333 0.94117647]
  [0.96078431 0.99215686 1.        ]
  ...
  [0.45882353 0.43921569 0.4627451 ]
  [0.14509804 0.1254902  0.15294118]
  [0.10588235 0.08627451 0.11372549]]

 [[0.2745098  0.28627451 0.30588235]
  [0.71764706 0.7372549  0.75294118]


In [8]:
print(x_train_norm[0])

[[[0.52941176 0.50196078 0.52941176]
  [0.5254902  0.49803922 0.5254902 ]
  [0.51764706 0.49411765 0.50980392]
  ...
  [0.10588235 0.0627451  0.03921569]
  [0.10980392 0.07058824 0.03529412]
  [0.14117647 0.10196078 0.0627451 ]]

 [[0.5372549  0.50980392 0.5372549 ]
  [0.53333333 0.50588235 0.53333333]
  [0.5254902  0.50196078 0.51764706]
  ...
  [0.11372549 0.07058824 0.04705882]
  [0.1254902  0.08627451 0.05098039]
  [0.17254902 0.13333333 0.09411765]]

 [[0.54509804 0.51764706 0.54509804]
  [0.54117647 0.51372549 0.54117647]
  [0.53333333 0.50980392 0.5254902 ]
  ...
  [0.13333333 0.09019608 0.06666667]
  [0.14509804 0.10588235 0.07058824]
  [0.2        0.16078431 0.12156863]]

 ...

 [[0.60392157 0.65882353 0.6627451 ]
  [0.61960784 0.6745098  0.67843137]
  [0.56078431 0.60784314 0.60784314]
  ...
  [0.45882353 0.44313725 0.43921569]
  [0.40784314 0.38823529 0.37254902]
  [0.35294118 0.3254902  0.30196078]]

 [[0.49803922 0.55294118 0.55686275]
  [0.60392157 0.65882353 0.65882353]
